Сначала размечаем данные train

In [3]:
import pandas as pd
from transformers import pipeline
import time


MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"
CATEGORIES = ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль',
                  'товары для детей',
                  'украшения и аксессуары',
                  'электроника',
                  'нет товара']



print("Загружаем модель...")
model = pipeline("text-generation", model=MODEL_NAME, device=-1)  # device=-1 для CPU


Загружаем модель...


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.17it/s]
Device set to use cpu


Читаем данные...
Будем размечать 5 отзывов
Начинаем разметку...

Готово! Общее время: 391.32 секунд
Среднее время на отзыв: 78.2642 секунд
Максимальное время: 95.8583 секунд
Отзывов дольше 5 секунд: 5

Распределение по категориям:
category
нет товара    3
одежда        2
Name: count, dtype: int64


In [6]:
INPUT_FILE = "train.csv"
OUTPUT_FILE = "ready_train.csv"


print("Читаем данные...")
df = pd.read_csv(INPUT_FILE)
reviews = df['text'].dropna().tolist()
print(f"Будем размечать {len(reviews)} отзывов")

Читаем данные...
Будем размечать 1168 отзывов


In [7]:
KEYWORDS = {
    "бытовая техника": [
        "холодильник", "стиральная", "посудомоечная", "микроволновка", "духовка",
        "плита", "кофемашина", "чайник", "блендер", "миксер", "пылесос",
        "фен", "мультиварка", "тостер", "вытяжка", "кондиционер", "обогреватель"
    ],
    "обувь": [
        "туфли", "ботинки", "сапоги", "кроссовки", "кеды", "сандали", "босоножки",
        "тапочки", "каблук", "подошва", "размер обуви", "мокасины", "лоферы",
        "угги", "берцы", "шлепанцы", "зимняя обувь", "летняя обувь"
    ],
    "одежда": [
        "футболка", "рубашка", "блузка", "свитер", "толстовка", "брюки", "джинсы",
        "штаны", "шорты", "юбка", "платье", "пиджак", "куртка", "пальто", "пуховик",
        "белье", "носки", "колготки", "спортивный костюм", "размер одежды",
         "трусы", "бюстгальтер", "майка", "носки", "колготки", "чулки",
         "леггинсы", "лосины",

        # Бренды и типы
        "джинсы", "свитшот", "топ", "боди", "кимоно", "туника", "бомбер", "парка",
        "блейзер", "кардиган", "пончо", "кепи", "кепка", "шапка", "шарф", "перчатки"
    ],
    "посуда": [
        "кастрюля", "сковорода", "тарелка", "чашка", "блюдо", "стакан", "кружка",
        "нож", "вилка", "ложка", "сковородка", "ковш", "сотейник", "чайник",
        "кофейник", "сервиз", "набор посуды", "столовые приборы"
    ],
    "текстиль": [
        "полотенце", "простыня", "пододеяльник", "наволочка", "покрывало", "скатерть",
        "салфетка", "шторы", "занавески", "ковер", "плед", "одеяло", "подушка",
        "белье постельное", "матрас", "чехол", "габардин", "сатин", "хлопок"
    ],
    "товары для детей": [
        "игрушка", "кукла", "машинка", "конструктор", "пазл", "коляска", "кроватка",
        "пеленальный", "соска", "бутылочка", "подгузники", "питание детское",
        "одежда детская", "обувь детская", "рюкзак школьный", "канцтовары",
        "мобиль", "радионяня"
    ],
    "украшения и аксессуары": [
        "кольцо", "серьги", "браслет", "цепочка", "кулон", "подвеска", "ожерелье",
        "бижутерия", "сумка", "рюкзак", "кошелек", "ремень", "галстук", "часы",
        "очки", "солнцезащитные", "зонт", "шапка", "шарф", "перчатки"
    ],
    "электроника": [
        "ноутбук", "планшет", "компьютер", "монитор",
        "телевизор", "наушники", "колонки", "фотоаппарат", "камера", "роутер",
        "мышка", "клавиатура", "powerbank", "зарядное", "аккумулятор"
    ]
}


print("Начинаем разметку...")
results = []
total_start = time.time()

for i, review in enumerate(reviews):
    start_time = time.time()


    category = "нет товара"
    review_lower = review.lower()


    priority_categories = ["одежда", "обувь", "украшения и аксессуары"] + [cat for cat in CATEGORIES if cat not in ["одежда", "обувь", "украшения и аксессуары", "нет товара"]]

    for cat in priority_categories:
        if cat in KEYWORDS:
            for keyword in KEYWORDS[cat]:
                if keyword.lower() in review_lower:
                    category = cat
                    break
            if category != "нет товара":
                break


    if category == "нет товара":
        prompt = f"Отнеси этот отзыв к одной из категорий ({', '.join(CATEGORIES)}): '{review[:100]}' Ответ:"
        response = model(prompt, max_new_tokens=4, temperature=0.1)
        answer = response[0]['generated_text'].replace(prompt, "").strip()

        for cat in CATEGORIES:
            if cat.lower() in answer.lower():
                category = cat
                break

    processing_time = time.time() - start_time

    results.append({
        "text": review,
        "category": category,
    })

    # Сохраняем каждые 100 отзывов
    if (i + 1) % 10 == 0:
        df_temp = pd.DataFrame(results)
        df_temp.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')
        print(f"Сохранено {i+1} отзывов")

    # Прогресс
    if (i + 1) % 10 == 0:
        print(f"Обработано {i+1}/{len(reviews)} отзывов")


df_results = pd.DataFrame(results)
df_results.to_csv(OUTPUT_FILE, index=False, encoding='utf-8')


total_time = time.time() - total_start
print(f"\nГотово! Общее время: {total_time:.2f} секунд")
print(f"Среднее время на отзыв: {df_results['processing_time'].mean():.4f} секунд")
print(f"Максимальное время: {df_results['processing_time'].max():.4f} секунд")

print("\nРаспределение по категориям:")
print(df_results['category'].value_counts())

Начинаем разметку...
Обработано 10/1168 отзывов
Обработано 20/1168 отзывов
Обработано 30/1168 отзывов
Обработано 40/1168 отзывов
Обработано 50/1168 отзывов
Обработано 60/1168 отзывов
Обработано 70/1168 отзывов
Обработано 80/1168 отзывов
Обработано 90/1168 отзывов
Сохранено 100 отзывов
Обработано 100/1168 отзывов
Обработано 110/1168 отзывов
Обработано 120/1168 отзывов
Обработано 130/1168 отзывов
Обработано 140/1168 отзывов
Обработано 150/1168 отзывов
Обработано 160/1168 отзывов
Обработано 170/1168 отзывов
Обработано 180/1168 отзывов
Обработано 190/1168 отзывов
Сохранено 200 отзывов
Обработано 200/1168 отзывов
Обработано 210/1168 отзывов
Обработано 220/1168 отзывов


KeyboardInterrupt: 

Генерация новых отзывов по категориям

In [ ]:
from transformers import pipeline

gen = pipeline("text-generation", model="microsoft/Phi-3-mini-4k-instruct")


results = {}

print("Начинаем генерацию...\n")

for cat in CATEGORIES:
    results[cat] = []

    for i in range(3):
        prompt = f"напиши короткий отзыв на товар с маркетплейса, товар категории: {cat} (1-2 предложения):"
        response = gen(prompt, max_new_tokens=25)

        # Быстрая очистка ответа
        review = response[0]['generated_text'].split(":")[-1].strip()


        # results[cat].append(review)

        df = pd.DataFrame([{'text': review, 'category' : cat}])
        df.to_csv('ready_train.csv', index=False, encoding='utf-8', mode='a', header=False)

print("ВСЕ ОТЗЫВЫ СГЕНЕРИРОВАНЫ!")



Обучение модели на размеченных данных

In [ ]:
import pandas as pd
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
from datasets import Dataset
import time

# Включим автоматическое смешанная точность (AMP) для PyTorch
torch.backends.cuda.matmul.allow_tf32 = True  # Разрешить TF32 для матричных умножений
torch.backends.cudnn.allow_tf32 = True        # Разрешить TF32 для cuDNN


df = pd.read_csv('ready_train.csv')
print(f"Загружено {len(df)} размеченных отзывов")


categories = CATEGORIES
num_labels = len(categories)
label2id = {label: idx for idx, label in enumerate(categories)}
id2label = {idx: label for idx, label in enumerate(categories)}
df['label'] = df['category'].map(label2id)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])


MODEL_NAME = "cointegrated/rubert-tiny2"


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=True,
        max_length=128,
    )

train_dataset = Dataset.from_pandas(train_df[['text', 'label']])
val_dataset = Dataset.from_pandas(val_df[['text', 'label']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)


model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    torch_dtype=torch.float16,  # Используем float16 вместо float32
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model = torch.compile(model)  # Компиляция модели для ускорения


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"weighted_f1": f1_score(labels, predictions, average='weighted')}


training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=32,  # Увеличили batch size
    per_device_eval_batch_size=64,   # Увеличили для валидации
    num_train_epochs=2,              # Уменьшили количество эпох
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="weighted_f1",
    greater_is_better=True,


    fp16=True,                       # Автоматическое смешанная точность
    tf32=True,                       # Использовать TensorFloat-32
    dataloader_pin_memory=True,      # Закрепление памяти для быстрой загрузки
    dataloader_num_workers=2,        # Многопоточная загрузка данных
    gradient_accumulation_steps=1,
    logging_steps=50,
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Начинаем обучение с оптимизацией...")
start_time = time.time()

trainer.train()

end_time = time.time()
print(f"Обучение заняло: {end_time - start_time:.2f} секунд")

# 12. Сохранение оптимизированной модели
trainer.save_model("./optimized_model")
tokenizer.save_pretrained("./optimized_model")
